In [106]:
from sympy import symbols, diff, solve, Symbol, Matrix
from sympy.abc import x, y

from scipy.optimize import newton, minimize, linprog

import numpy as np
import cvxpy
from pulp import *

# **Метод Ньютона**

### Поиск корней уравнения

**Задание 3.1**

Найдите третий корень полинома $f(x) = 6x^5 - 5x^4 - 4x^3 +3x^2$, взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

In [2]:
def func_x(x):
    return 6*x**5 - 5*x**4 - 4*x**3 +3*x**2

def diff_x(x):
    return 30*x**4 - 20*x**3 - 12*x**2 + 6*x

In [3]:
x0 = 0.7
x_cur = x0
i = 1
while abs(func_x(x_cur)) > 0.00000001:
    x_new = x_cur - func_x(x_cur)/diff_x(x_cur)
    print(i, ' ', round(x_cur, 3), ' ', round(x_new, 3), func_x(x_new))
    x_cur = x_new
    i += 1

1   0.7   0.63 -0.0012133284487552132
2   0.63   0.629 -1.3785387997788945e-06
3   0.629   0.629 -1.8043344596208044e-12


### Оптимизация функции

#### Дана функция $f(x) = x^3 - 3x^2 - 45x + 40$

In [7]:
f = x**3 - 3*x**2 - 45*x + 40
display(f.diff(x))
display(f.diff(x, 2))

3*x**2 - 6*x - 45

6*(x - 1)

In [8]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

In [9]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


#### Можно объединить всё в одну функцию

In [10]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)

5.0

#### можно воспользоваться реализацией из библиотеки scipy

In [12]:
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

## **NB: Метод Ньютона стоит применять только для задач, в которых целевая функция выпуклая**

**Задание 3.6**

Дана функция $f(x) = x^3-72x-220$. Найдите решение уравнения $f(x) = 0$ для поиска корня в окрестностях точки $x_0=12$

In [15]:
def f_3_6(x):
    return x**3 - 72*x - 220

def fx_3_6(x):
    return 3*x**2 - 72

round(newton(func=f_3_6, fprime=fx_3_6, x0=12, tol=0.00001), 3)

9.727

**Задание 3.7**

Найдите положительный корень для уравнения $x^2+9x-5 = 0$.

В качестве стартовой точки возьмите $x_0=2.2$.

In [17]:
def f_3_7(x):
    return x**2 + 9*x - 5

def fx_3_7(x):
    return 2*x + 9
res = newton(func=f_3_7, fprime=fx_3_7, x0=2.2, tol=0.00001)
if res > 0:
    print(round(res, 2))

0.52


**Задание 3.9**

С помощью метода Ньютона найдите точку минимума для функции $f(x)=8x^3-2x^2-450$.

В качестве стартовой точки возьмите $42$, точность примите за $0.0001$.

In [47]:
x = Symbol('x')
f_3_8 = 8*x**3-2*x**2-450
fx_3_8 = f_3_8.diff(x)
fxx_3_8 = f_3_8.diff(x, 2)

In [48]:
def dpime(x1):
    return fx_3_8.subs({x: x1})

def dsec(x2):
    return fxx_3_8.subs({x: x2})

In [41]:
dpime()

42


24*x**2 - 4*x

In [25]:
fx_3_8.subs({x: 42})

42168

In [49]:
x_cur = 42
eps = 0.0001

while dpime(x_cur) > eps:
    x_cur = x_cur - dpime(x_cur)/dsec(x_cur)

print(round(x_cur, 3))

0.167


# **Квазиньютоновские методы**

## Cхема Бройдена — Флетчера — Гольдфарба — Шанно (BFGS)

Cамая известная, стабильная и наиболее эффективная схема апроксимации гессина, созданная Чарли Джорджем Бройденом, Роджером Флетчером, Дональдом Гольдфарбом и Дэвидом Шанно.

У этой схемы есть две известных вариации:

- **L-BFGS**
- **L-BFGS-B**

**Пример 1**

Будем искать экстремум для функции следующего вида:

$f(x,y)=x^2-xy+y^2+9x-6y+20$

В качестве начальной точки выберем $x_0=(1,1)$, а точность - $0.001$

In [55]:
f = x**2 - x*y + y**2 + 9*x - 6*y + 20
fx = f.diff(x)
fy = f.diff(y)
df = Matrix([fx, fy])

In [62]:
fy

-x + 2*y - 6

In [57]:
df.subs({x: 1, y: 1}).norm()

5*sqrt(5)

**Пример 2**

Оптимизация функции $f(x,y)=x^2+y^2$

#### BFGS

In [59]:
# Определим функцию, которую будем оптимизировать. Вместо отдельных  и  можно взять координаты единого вектора:
def func(x):
    return x[0]**2.0 + x[1]**2.0

# Jпределим градиент для функции:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

# Задаём начальную точку
x_0 = [1.0, 1.0]

# Определяем алгоритм
result = minimize(func, x_0, method='BFGS', jac=grad_func)

# Выводим результаты
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


#### L-BFGS-B

In [60]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


**Задание 4.1**

Найдите точку минимума для функции $f(x,y)=x^2-xy+y^2+9x-6y+20$.

В качестве стартовой возьмите точку $(-400, -400)$

In [68]:
def f_4_1(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

def g_4_1(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

x_0 = [-400, -400]
res_4_1 = minimize(f_4_1, x_0, method='L-BFGS-B', jac=g_4_1)

solution = res_4_1['x']
evaluation = f_4_1(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Решение: f([-3.99999972  1.00000028]) = -1.00000


**Задание 4.4**

Найдите минимум функции $f(x)=x^3-3x+45$ с помощью квазиньютоновского метода BFGS.

В качестве стартовой точки возьмите $x=10$.

In [90]:
def f_4_4(x):
    return x**2 - 3*x + 45

def g_4_4(x):
    return np.array(2*x - 3)

x_0 = 10
res_4_4 = minimize(f_4_4, x_0, method='BFGS', jac=g_4_4)

solution = res_4_4['x']
evaluation = f_4_4(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
print('Статус оптимизации %s' % res_4_4['message'])
print('Количество оценок: %d' % res_4_4['nfev'])

Решение: f([1.5]) = 42.75000
Статус оптимизации Optimization terminated successfully.
Количество оценок: 5


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2432\1125334339.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Решение: f(%s) = %.5f' % (solution, evaluation))


**Задание 4.5**

Оптимизировать функцию из задания 4.4 методом L-BFGS-B

In [74]:
def f_4_5(x):
    return x**2 - 3*x + 45

def g_4_5(x):
    return 2*x - 3

x_0 = 10
res_4_5 = minimize(f_4_5, x_0, method='L-BFGS-B', jac=g_4_5)

solution = res_4_5['x']
evaluation = f_4_5(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
print('Статус оптимизации %s' % res_4_5['message'])
print('Количество оценок: %d' % res_4_5['nfev'])

Решение: f([1.5]) = 42.75000
Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2432\2161491471.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Решение: f(%s) = %.5f' % (solution, evaluation))


**Задание 4.7**

Найдите минимум функции
$$f(x,y)=x^4+6y^2+10$$
взяв за стартовую точку $(100,100)$.

In [87]:
def f_4_7(x):
    return x[0]**4.0 + 6*x[1]**2.0 + 10
 
def g_4_7(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
x_0 = [100.0, 100.0]
res_4_7_1 = minimize(f_4_7, x_0, method='BFGS', jac=g_4_7)
res_4_7_2 = minimize(f_4_7, x_0, method='L-BFGS-B', jac=g_4_7)

print('Количество оценок BFGS: %d' % res_4_7_1['nfev'])
print('Количество оценок L-BFGS-B: %d' % res_4_7_2['nfev'])
print(f'Решение {round(f_4_7(res_4_7_1["x"]))}')

Количество оценок BFGS: 37
Количество оценок L-BFGS-B: 40
Решение 10


# **Линейное программирование**

**Пример 1** 

### SciPy (scipy.optimize.linprog)

У нас есть 6 товаров с заданными ценами на них и заданной массой. Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг. Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [88]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [92]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

# Передаём подготовленные переменные в оптимизатор SciPy

linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

**Пример 2**

### CVXPY

Снова решим задачу из примера 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.
SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY.

In [99]:
x = cvxpy.Variable(shape=n, integer=True) # только целое количество
constraint = (A @ x <= b) # задаём ограничение по суммарному весу
x_positive = (x >= 0) # указываем неотрицателньость количества
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
display(-problem.solve())
display(x.value)

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\cvxpy\expressions\expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, UserWarning)


49.0

array([-0., -0., -0.,  7., -0.,  0.])

In [98]:
x = cvxpy.Variable(shape=n, boolean=True) # 0 или 1 - берём только один товар или не берём вовсе
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
display(-problem.solve())
display(x.value)

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\cvxpy\expressions\expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


17.0

array([1., 0., 0., 1., 0., 1.])

**Пример 3**
### PuLP

В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

- Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
- Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
- Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней —  автомобиль B.
- Каждый специалист может работать суммарно 30 дней.

In [108]:
problem = LpProblem('Производство_машин', LpMaximize)
A = LpVariable('Автомобиль_A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль_B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


**Задание 6.1**

Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e92f1aca3294f87b96e30eed2471af02/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md6_6_1_1.png.png)